In [6]:
#과제2 : 서브페이지 정보 크롤링 - 메뉴가격, 카페주소, 전화번호, 홈페이지import pandas as pd
import time
import re
import pandas as pd
from bs4 import BeautifulSoup 
from urllib.request import urlopen

## STEP5 : 메인페이지 크롤링한 데이터 파일 읽어오기
- 불필요한 칼럼 삭제
- 랭킹정보 인덱스 설정

In [7]:
df = pd.read_csv('data/chicagomag_info.csv', encoding='utf-8')
df.head()

,Unnamed: 0,랭킹,메뉴,카페,링크
0,0,1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,1,2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,2,3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,3,4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,4,5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [8]:
# index_col='Unnamed: 0'') 삭제
del df['Unnamed: 0']
df.head()

,랭킹,메뉴,카페,링크
0,1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [9]:
df.set_index('랭킹', inplace=True)
df.head()

,메뉴,카페,링크
랭킹,,,
1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 50
Data columns (total 3 columns):
메뉴    50 non-null object
카페    50 non-null object
링크    50 non-null object
dtypes: object(3)
memory usage: 1.6+ KB


In [11]:
df['메뉴'][1]

'BLT'

In [12]:
df['링크'][1]

'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

### STEP6 : 서브페이지의 수집할 데이터 Tag & Class 확인
## 서브페이지 크롤링대상 타겟데이터

- 메뉴가격
- 카페주소
- 전화번호
- 홈페이지

In [14]:
site = df['링크'][1]
html = urlopen(site)
soup = BeautifulSoup(html, "lxml")

In [15]:
# 서브페이지 로딩 확인
soup.title

<title>
  1. Old Oak Tap BLT |
  Chicago magazine
      |  November 2012
    </title>

In [16]:
p_tag = soup.find('p', 'addy')
p_tag

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [17]:
print(p_tag.prettify())

<p class="addy">
 <em>
  $10. 2109 W. Chicago Ave., 773-772-0406,
  <a href="http://www.theoldoaktap.com/">
   theoldoaktap.com
  </a>
 </em>
</p>



In [18]:
taginfo = p_tag.get_text()
taginfo

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [19]:
# taginfo = taginfo[taginfo.find('$'):]
taginfo = taginfo.strip()
taginfo

'$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

### STEP7 : RegEx를 활용하여 분류
## 태그정보로는 분류 안됨

- 메뉴가격 : $로 시작
- 카페주소 : 패턴 안보이므로 일단보류. 가격과 전화번호 사이
- 전화번호 : 이거슨 RegEx, 숫자3-숫자3-숫자4
- 홈페이지 : 요것도 RegEx, .com 혹은 .net 으로 끝남

### 참조. Regular Expression 관련 사이트
## http://regexr.com/ : text 정보를 re로 테스트
## https://regexper.com/ : 작성된 re를 다이어그램으로 표현

In [20]:
# 메뉴가격
menu_price = re.search('\$\d+\.(\d+)?', taginfo).group()
menu_price

'$10.'

In [21]:
# 카페주소
cafe_addr = '일단보류' 

In [22]:
# 전화번호
cafe_tel = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
if cafe_tel is not None:
    cafe_tel = cafe_tel.group()
else:
    cafe_tel = 'No Contact'
    
cafe_tel

'773-772-0406'

In [23]:
taginfo

'$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [24]:
info_list = taginfo.split(', ')
info_list

['$10. 2109 W. Chicago Ave.', '773-772-0406', 'theoldoaktap.com']

In [25]:
# 홈페이지
if info_list[-1].endswith('.com') or info_list[-1].endswith('.net'):
    home_page = info_list[-1]
else:
    home_page = 'No Homepage'

home_page

'theoldoaktap.com'

In [26]:
info_list[0]

'$10. 2109 W. Chicago Ave.'

In [27]:
menu_price

'$10.'

In [28]:
# 카페주소
idx = len(menu_price)
cafe_addr = info_list[0][idx:].strip()
cafe_addr

'2109 W. Chicago Ave.'

## STEP8 : 서브페이지 정보 크롤링 모듈
- 파일의 링크정보에 순차적으로 접속하여 데이터 크롤링
- 메뉴가격, 카페주소, 전화번호, 홈페이지
- 우선 TOP5 페이지정도만 수집후 데이터 검토

In [29]:
for ranking in df.index[:5]:
    sub_link = df['링크'][ranking]
    print('Top{} : {}'.format(ranking, sub_link))

Top1 : http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/
Top2 : http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/
Top3 : http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/
Top4 : http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/
Top5 : http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/


In [30]:
price_list = list()
cfadd_list = list()
cftel_list = list()
homep_list = list()

for ranking in df.index[:5]:
    sub_link = df['링크'][ranking]
    # print('{} : {}'.format(idx, sub_link))
    html = urlopen(sub_link)
    soup = BeautifulSoup(html, "lxml")
    
    p_tag = soup.find('p', 'addy')
    taginfo = p_tag.get_text()
    taginfo = taginfo.strip()

    # 메뉴가격
    menu_price = re.search('\$\d+\.(\d+)?', taginfo).group()

    # 카페주소
    cafe_addr = '일단보류' 

    # 전화번호
    cafe_tel = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
    if cafe_tel is not None:
        cafe_tel = cafe_tel.group()
    else:
        cafe_tel = 'No Contact'

    info_list = taginfo.split(', ')

    # 홈페이지
    if info_list[-1].endswith('.com') or info_list[-1].endswith('.net'):
        home_page = info_list[-1]
    else:
        home_page = 'No Homepage'

    # 카페주소
    idx = len(menu_price)
    cafe_addr = info_list[0][idx:].strip()

    price_list.append(menu_price)
    cfadd_list.append(cafe_addr )
    cftel_list.append(cafe_tel  )
    homep_list.append(home_page )
    
    print('Top{} Page : Crawling is completed.'.format(ranking))
else:
    print('-'*50)
    print('크롤링 완료!!!')

Top1 Page : Crawling is completed.
Top2 Page : Crawling is completed.
Top3 Page : Crawling is completed.
Top4 Page : Crawling is completed.
Top5 Page : Crawling is completed.
--------------------------------------------------
크롤링 완료!!!


In [31]:
price_list

['$10.', '$9.', '$9.50', '$9.40', '$10.']

In [32]:
cfadd_list

['2109 W. Chicago Ave.',
 '800 W. Randolph St.',
 '. 445 N. Clark St.',
 '. 914 Noyes St.',
 '825 W. Fulton Mkt.']

In [33]:
cftel_list

['773-772-0406',
 '312-929-4580',
 '312-334-3688',
 '847-475-9400',
 '312-445-8977']

In [34]:
homep_list

['theoldoaktap.com',
 'aucheval.tumblr.com',
 'rickbayless.com',
 'alsdeli.net',
 'publicanqualitymeats.com']

In [35]:
price_list

['$10.', '$9.', '$9.50', '$9.40', '$10.']

# 메뉴가격과 카페주소 정보를 수정보완

In [36]:
menu_price = price_list[0]
if menu_price.endswith('.'):
    menu_price = menu_price[:-1]
menu_price

'$10'

In [37]:
cafe_addr = cfadd_list[2]
if cafe_addr.startswith('.'):
    cafe_addr = cafe_addr[1:].strip()
cafe_addr

'445 N. Clark St.'

In [38]:
# 만약 문제가 없다면, 전체 정보를 한번에 다 가져오기

from tqdm import tqdm_notebook

def getSubpagesData(df):

    ret_dict  = dict()
    columns   = [ '메뉴가격', '카페주소', '전화번호', '홈페이지' ]
    price_list = list()
    cfadd_list = list()
    cftel_list = list()
    homep_list = list()

    for ranking in tqdm_notebook(df.index):
        
        sub_link = df['링크'][ranking]
        html = urlopen(sub_link)
        soup = BeautifulSoup(html, "lxml")

        p_tag = soup.find('p', 'addy')
        taginfo = p_tag.get_text()
        taginfo = taginfo.strip()

        # 메뉴가격
        menu_price = re.search('\$\d+\.(\d+)?', taginfo).group()

        # 카페주소
        # cafe_addr = '일단보류' 

        # 전화번호
        cafe_tel = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
        if cafe_tel is not None:
            cafe_tel = cafe_tel.group()
        else:
            cafe_tel = 'No Contact'

        info_list = taginfo.split(', ')

        # 홈페이지
        if info_list[-1].endswith('.com') or info_list[-1].endswith('.net'):
            home_page = info_list[-1]
        else:
            home_page = 'No Homepage'

        # 카페주소
        idx = len(menu_price)
        cafe_addr = info_list[0][idx:].strip()
        
        # 리펙토링
        menu_price = menu_price[:-1] if menu_price.endswith('.') else menu_price
        cafe_addr  = cafe_addr[1:].strip() if cafe_addr.startswith('.') else cafe_addr
        
        # if menu_price.endswith('.'):
        #     menu_price = menu_price[:-1]
        # if cafe_addr.startswith('.'):
        #     cafe_addr = cafe_addr[1:].strip()
        

        price_list.append(menu_price)
        cfadd_list.append(cafe_addr )
        cftel_list.append(cafe_tel  )
        homep_list.append(home_page )

        # print('Ranking{} subpage crawling is completed.'.format(ranking))
        separator = ' |\n' if ranking%5==0 else ' | '
        print('Top{0:2d} 완료'.format(ranking), end=separator)
    else:
        print(); print('-'*100);
        print('모든 서브페이지 크롤링 완료!!!')

    ret_dict[columns[0]] = price_list
    ret_dict[columns[1]] = cfadd_list
    ret_dict[columns[2]] = cftel_list
    ret_dict[columns[3]] = homep_list
    
    return ret_dict

In [39]:
ret_dict = getSubpagesData(df)

Top 1 완료 | Top 2 완료 | Top 3 완료 | Top 4 완료 | Top 5 완료 |
Top 6 완료 | Top 7 완료 | Top 8 완료 | Top 9 완료 | Top10 완료 |
Top11 완료 | Top12 완료 | Top13 완료 | Top14 완료 | Top15 완료 |
Top16 완료 | Top17 완료 | Top18 완료 | Top19 완료 | Top20 완료 |
Top21 완료 | Top22 완료 | Top23 완료 | Top24 완료 | Top25 완료 |
Top26 완료 | Top27 완료 | Top28 완료 | Top29 완료 | Top30 완료 |
Top31 완료 | Top32 완료 | Top33 완료 | Top34 완료 | Top35 완료 |
Top36 완료 | Top37 완료 | Top38 완료 | Top39 완료 | Top40 완료 |
Top41 완료 | Top42 완료 | Top43 완료 | Top44 완료 | Top45 완료 |
Top46 완료 | Top47 완료 | Top48 완료 | Top49 완료 | Top50 완료 |


----------------------------------------------------------------------------------------------------
모든 서브페이지 크롤링 완료!!!


In [40]:
ret_dict.keys()

dict_keys(['메뉴가격', '카페주소', '전화번호', '홈페이지'])

In [41]:
ret_dict['메뉴가격']

['$10',
 '$9',
 '$9.50',
 '$9.40',
 '$10',
 '$7.25',
 '$16',
 '$10',
 '$9',
 '$17',
 '$11',
 '$5.49',
 '$14',
 '$10',
 '$13',
 '$4.50',
 '$11.95',
 '$11.50',
 '$6.25',
 '$15',
 '$5',
 '$6',
 '$8',
 '$5.99',
 '$7.52',
 '$11.95',
 '$7.50',
 '$12.95',
 '$7',
 '$21',
 '$9.79',
 '$9.75',
 '$13',
 '$7.95',
 '$9',
 '$9',
 '$8',
 '$8',
 '$7',
 '$6',
 '$7.25',
 '$11',
 '$6',
 '$9',
 '$5.49',
 '$8',
 '$6.50',
 '$7.50',
 '$8.75',
 '$6.85']

In [42]:
ret_dict['카페주소']

['2109 W. Chicago Ave.',
 '800 W. Randolph St.',
 '445 N. Clark St.',
 '914 Noyes St.',
 '825 W. Fulton Mkt.',
 '100 E. Walton St.',
 '1639 S. Wabash Ave.',
 '2211 W. North Ave.',
 '3619 W. North Ave.',
 '3267 S. Halsted St.',
 '2537 N. Kedzie Blvd.',
 'Multiple locations',
 '3124 N. Broadway',
 '3455 N. Southport Ave.',
 '2657 N. Kedzie Ave.',
 '1120 W. Grand Ave.',
 '1141 S. Jefferson St.',
 '333 E. Benton Pl.',
 '1411 N. Wells St.',
 '1747 N. Damen Ave.',
 '3209 W. Irving Park Rd.',
 'Multiple locations',
 '5347 N. Clark St.',
 '2954 W. Irving Park Rd.',
 'Multiple locations',
 '191 Skokie Valley Rd.',
 'Multiple locations',
 '1818 W. Wilson Ave.',
 '2517 W. Division St.',
 '218 W. Kinzie St.',
 'Multiple locations',
 '1547 N. Wells St.',
 '415 N. Milwaukee Ave.',
 '1840 N. Damen Ave.',
 '1220 W. Webster Ave.',
 '5357 N. Ashland Ave.',
 '1834 W. Montrose Ave.',
 '615 N. State St.',
 'Multiple locations',
 '241 N. York Rd.',
 '1323 E. 57th St.',
 '655 Forest Ave.',
 'Hotel Lincoln',


In [43]:
ret_dict['전화번호']

['773-772-0406',
 '312-929-4580',
 '312-334-3688',
 '847-475-9400',
 '312-445-8977',
 '312-649-6717',
 '312-360-9500',
 '773-276-2100',
 '773-772-8435',
 '312-929-2486',
 '773-489-9554',
 'No Contact',
 '773-661-9166',
 '773-883-2525',
 '773-276-7110',
 '312-666-0730',
 '312-939-2855',
 '773-234-3449',
 '312-944-0459',
 '773-489-1747',
 '773-539-8038',
 'No Contact',
 '773-275-5725',
 '773-539-5321',
 'No Contact',
 '847-831-0600',
 'No Contact',
 '773-293-2489',
 '773-862-8313',
 '312-624-8154',
 'No Contact',
 '312-624-9430',
 '312-829-6300',
 '773-681-9914',
 '773-883-1313',
 '773-275-4297',
 '773-334-5664',
 '312-265-0434',
 'No Contact',
 '630-516-3354',
 '773-538-7372',
 '847-234-8800',
 '312-254-4665',
 '708-725-7200',
 '312-922-2233',
 '773-384-9930',
 '312-896-2600',
 '630-359-5234',
 '312-726-2020',
 '773-868-4000']

In [44]:
ret_dict['홈페이지']

['theoldoaktap.com',
 'aucheval.tumblr.com',
 'rickbayless.com',
 'alsdeli.net',
 'publicanqualitymeats.com',
 'No Homepage',
 'acadiachicago.com',
 'birchwoodkitchen.com',
 'cemitaspuebla.com',
 'nanaorganic.com',
 'lulacafe.com',
 'ricobenespizza.com',
 'frognsnail.com',
 'crosbyskitchenchicago.com',
 'longmanandeagle.com',
 'bariitaliansubs.com',
 'mannysdeli.com',
 'eggysdiner.com',
 'oldjerusalemchicago.com',
 'hotchocolatechicago.com',
 'No Homepage',
 'dawalikitchen.com',
 'bigjoneschicago.com',
 'lapanechicago.com',
 'pastoralartisan.com',
 'maxs-deli.com',
 'luckysandwich.com',
 'cityprovisions.com',
 'papascachesabroso.com',
 'No Homepage',
 'hannahsbretzel.com',
 'lafournette.com',
 'paramountroom.com',
 'meltsandwichshoppechicago.com',
 'floriole.com',
 'No Homepage',
 'troquetchicago.com',
 'grahamwich.com',
 'saigonsisters.com',
 'rosaliasdeli.com',
 'zhmarketcafe.com',
 'themarkethouse.com',
 'No Homepage',
 'marionstreetcheesemarket.com',
 'cafecitochicago.com',
 'chick

### STEP9 : 최종 크롤링 데이터 저장
## 기존 데이터에 칼럼 추가
- 메인페이지 정보 크롤링 - 랭킹, 메뉴, 카페, 링크(상세페이지)
- 서브페이지 정보 크롤링 - 메뉴가격, 카페주소, 전화번호, 홈페이지
- 컬럼순서 조정후 저장

In [45]:
df.head()

,메뉴,카페,링크
랭킹,,,
1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [46]:
# 기존 데이터에 칼럼 추가하기
columns = [ '메뉴가격', '카페주소', '전화번호', '홈페이지' ]
# df['메뉴가격'] = ret_dict['메뉴가격']
# df['카페주소'] = ret_dict['카페주소']
# df['전화번호'] = ret_dict['전화번호']
# df['홈페이지'] = ret_dict['홈페이지']

for column in columns:
    df[column] = ret_dict[column]

In [47]:
df.head()

,메뉴,카페,링크,메뉴가격,카페주소,전화번호,홈페이지
랭킹,,,,,,,
1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,2109 W. Chicago Ave.,773-772-0406,theoldoaktap.com
2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9,800 W. Randolph St.,312-929-4580,aucheval.tumblr.com
3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.50,445 N. Clark St.,312-334-3688,rickbayless.com
4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.40,914 Noyes St.,847-475-9400,alsdeli.net
5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,825 W. Fulton Mkt.,312-445-8977,publicanqualitymeats.com


In [48]:
# loc 혹은 iloc 를 사용하면 컬럼과 레코드전체가 옮겨진다. 
new_columns = [ '카페', '메뉴', '메뉴가격', '전화번호', '카페주소', '홈페이지', '링크' ]

In [49]:
df = df.loc[:, new_columns]
df.head()

,카페,메뉴,메뉴가격,전화번호,카페주소,홈페이지,링크
랭킹,,,,,,,
1,Old Oak Tap,BLT,$10,773-772-0406,2109 W. Chicago Ave.,theoldoaktap.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,Au Cheval,Fried Bologna,$9,312-929-4580,800 W. Randolph St.,aucheval.tumblr.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,Xoco,Woodland Mushroom,$9.50,312-334-3688,445 N. Clark St.,rickbayless.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,Al’s Deli,Roast Beef,$9.40,847-475-9400,914 Noyes St.,alsdeli.net,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,Publican Quality Meats,PB&L,$10,312-445-8977,825 W. Fulton Mkt.,publicanqualitymeats.com,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 50
Data columns (total 7 columns):
카페      50 non-null object
메뉴      50 non-null object
메뉴가격    50 non-null object
전화번호    50 non-null object
카페주소    50 non-null object
홈페이지    50 non-null object
링크      50 non-null object
dtypes: object(7)
memory usage: 5.6+ KB


In [51]:
# 추가로 변경작업한 데이터 파일로 저장하기 
df.to_csv('data/chicagomag_info_ver2.csv', encoding='UTF-8')

In [52]:
# 변경저장된 데이터 파일 확인하기 
final_df = pd.read_csv('data/chicagomag_info_ver2.csv', index_col='랭킹', encoding='UTF-8')
final_df.head()

,카페,메뉴,메뉴가격,전화번호,카페주소,홈페이지,링크
랭킹,,,,,,,
1,Old Oak Tap,BLT,$10,773-772-0406,2109 W. Chicago Ave.,theoldoaktap.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,Au Cheval,Fried Bologna,$9,312-929-4580,800 W. Randolph St.,aucheval.tumblr.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,Xoco,Woodland Mushroom,$9.50,312-334-3688,445 N. Clark St.,rickbayless.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,Al’s Deli,Roast Beef,$9.40,847-475-9400,914 Noyes St.,alsdeli.net,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,Publican Quality Meats,PB&L,$10,312-445-8977,825 W. Fulton Mkt.,publicanqualitymeats.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
